In [1]:
import numpy as np
from matplotlib import pyplot as plt
import h5py
import os
import datetime
import json

import geopandas as gpd
import pandas as pd

## Label dataframe

In [2]:
data_dir = '/work/ka1176/shared_data/2021-ai4food/raw_data/'

south_africa_tr_labels_dir_1 = os.path.join(data_dir, 'ref_fusion_competition_south_africa_train_labels/ref_fusion_competition_south_africa_train_labels_34S_19E_258N/labels.geojson')
south_africa_tr_labels_dir_2 = os.path.join(data_dir, 'ref_fusion_competition_south_africa_train_labels/ref_fusion_competition_south_africa_train_labels_34S_19E_259N/labels.geojson')
south_africa_te_labels_dir = os.path.join(data_dir, 'ref_fusion_competition_south_africa_test_labels/ref_fusion_competition_south_africa_test_labels_34S_20E_259N/labels.geojson')

In [7]:
labels_1 = gpd.read_file(south_africa_tr_labels_dir_1)
labels_2 = gpd.read_file(south_africa_tr_labels_dir_2)
labels_3 = gpd.read_file(south_africa_te_labels_dir)

n_train = len(labels_1) + len(labels_2)
print(n_train)

4151


In [8]:
combined_labels = pd.concat([labels_1, labels_2, labels_3])

shape_area_max = combined_labels['SHAPE_AREA'].iloc[:n_train].max()
shape_area_min = combined_labels['SHAPE_AREA'].iloc[:n_train].min()

normalized_shape_area = (combined_labels['SHAPE_AREA'] - shape_area_min) / (shape_area_max - shape_area_min)

shape_len_max = combined_labels['SHAPE_LEN'].max()
shape_len_min = combined_labels['SHAPE_LEN'].min()

normalized_shape_len  = (combined_labels['SHAPE_LEN'] - shape_len_min) / (shape_len_max - shape_len_min)

combined_labels['NORMALIZED_SHAPE_AREA'] = normalized_shape_area
combined_labels['NORMALIZED_SHAPE_LEN']  = normalized_shape_len


jf = '/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/labels_combined.geojson'

combined_labels.to_file(jf, driver="GeoJSON")
    
print(combined_labels.head())

     fid     SHAPE_AREA    SHAPE_LEN  crop_id       crop_name  \
0  80172   86608.783197  2165.333019        4  Lucerne/Medics   
1  80217  129169.262104  1478.177480        2          Barley   
2  80225  231420.387803  2007.104974        4  Lucerne/Medics   
3  80228  100817.991529  1468.847742        4  Lucerne/Medics   
4  80229  375263.554123  2693.791755        3          Canola   

                                            geometry  NORMALIZED_SHAPE_AREA  \
0  MULTIPOLYGON (((456686.552 6215911.790, 456700...               0.066863   
1  MULTIPOLYGON (((459030.598 6215734.313, 458988...               0.099723   
2  MULTIPOLYGON (((458356.570 6216251.177, 458489...               0.178669   
3  MULTIPOLYGON (((459818.661 6216018.564, 459809...               0.077834   
4  MULTIPOLYGON (((459945.661 6215862.459, 460150...               0.289726   

   NORMALIZED_SHAPE_LEN  
0              0.250336  
1              0.170389  
2              0.231927  
3              0.169304  
4   

In [11]:
data_dir = '/work/ka1176/shared_data/2021-ai4food/raw_data/'

ger_tr_labels_dir_1 = os.path.join(data_dir, 'dlr_fusion_competition_germany_train_labels/dlr_fusion_competition_germany_train_labels_33N_18E_242N/labels.geojson')
ger_te_labels_dir = os.path.join(data_dir, 'dlr_fusion_competition_germany_test_labels/dlr_fusion_competition_germany_test_labels_33N_17E_243N/labels.geojson')

In [12]:
labels_1 = gpd.read_file(ger_tr_labels_dir_1)
labels_3 = gpd.read_file(ger_te_labels_dir)

n_train = len(labels_1) 
print(n_train)

2534


In [13]:
combined_labels = pd.concat([labels_1, labels_3])

shape_area_max = combined_labels['SHAPE_AREA'].iloc[:n_train].max()
shape_area_min = combined_labels['SHAPE_AREA'].iloc[:n_train].min()

normalized_shape_area = (combined_labels['SHAPE_AREA'] - shape_area_min) / (shape_area_max - shape_area_min)

shape_len_max = combined_labels['SHAPE_LEN'].max()
shape_len_min = combined_labels['SHAPE_LEN'].min()

normalized_shape_len  = (combined_labels['SHAPE_LEN'] - shape_len_min) / (shape_len_max - shape_len_min)

combined_labels['NORMALIZED_SHAPE_AREA'] = normalized_shape_area
combined_labels['NORMALIZED_SHAPE_LEN']  = normalized_shape_len


jf = '/work/ka1176/shared_data/2021-ai4food/dev_data/germany/labels_combined.geojson'

combined_labels.to_file(jf, driver="GeoJSON")
    
print(combined_labels.head())

    fid    SHAPE_AREA    SHAPE_LEN  crop_id     crop_name  \
0    54    4400.70910   293.870062        9  Forage Crops   
1  3935  299598.99930  2574.818018        1         Wheat   
2  4073  194110.48135  1913.745250        1         Wheat   
3  4724   15059.00790  1181.569204        8       Meadows   
4  5336   75017.01935  1506.996389        2           Rye   

                                            geometry  NORMALIZED_SHAPE_AREA  \
0  MULTIPOLYGON (((449042.600 5827711.200, 449082...               0.002278   
1  MULTIPOLYGON (((455395.090 5828098.210, 455395...               0.159524   
2  MULTIPOLYGON (((452593.690 5827525.940, 452580...               0.103332   
3  MULTIPOLYGON (((433352.160 5819538.890, 433376...               0.007955   
4  MULTIPOLYGON (((454516.350 5826887.890, 454531...               0.039894   

   NORMALIZED_SHAPE_LEN  
0              0.017887  
1              0.176698  
2              0.130671  
3              0.079693  
4              0.102351  


## Sentinel-1, 640

In [3]:
assert False # stop automatic execution

In [3]:
train_S1_640 = h5py.File('/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/sentinel-1/extracted-640/train_data.h5', 'r')

In [7]:
samples = len(train_P_640['label'])
print('Samples before inflation:', samples)

h5_keys = train_S1_640.keys()
print(h5_keys)

Samples before inflation: 4143
<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>


In [34]:
target_size = 64

In [45]:
raw_ds = {key:[] for key in h5_keys}

for i in range(samples):
    this_crop_name = train_S1_640['crop_name'][i]
    this_fid       = train_S1_640['fid'][i]
    this_image_stack = train_S1_640['image_stack'][i]
    this_label       = train_S1_640['label'][i]
    this_mask        = train_S1_640['mask'][i]
    
    current_size = this_image_stack.shape[-1]
    
    inflation_factor = current_size // target_size
    
    print('Inflation factor', inflation_factor)
    
    for j in range(inflation_factor):
        # always the same
        raw_ds['crop_name'].append(this_crop_name) 
        raw_ds['fid'].append(this_fid)
        raw_ds['label'].append(this_label)
        # divide
        raw_ds['image_stack'].extend(this_image_stack[np.newaxis, :, :, j*target_size:(j+1)*target_size])
        raw_ds['mask'].extend(this_mask[np.newaxis, j*target_size:(j+1)*target_size])
    
    if i%100:
        print('Progress: ', i/samples * 100)

Inflation factor 10
Inflation factor 10
Progress:  0.02413709872073377
Inflation factor 10
Progress:  0.04827419744146754
Inflation factor 10
Progress:  0.0724112961622013
Inflation factor 10
Progress:  0.09654839488293508
Inflation factor 10
Progress:  0.12068549360366883
Inflation factor 10
Progress:  0.1448225923244026
Inflation factor 10
Progress:  0.1689596910451364
Inflation factor 10
Progress:  0.19309678976587016
Inflation factor 10
Progress:  0.2172338884866039
Inflation factor 10
Progress:  0.24137098720733766
Inflation factor 10
Progress:  0.26550808592807146
Inflation factor 10
Progress:  0.2896451846488052
Inflation factor 10
Progress:  0.313782283369539
Inflation factor 10
Progress:  0.3379193820902728
Inflation factor 10
Progress:  0.3620564808110065
Inflation factor 10
Progress:  0.3861935795317403
Inflation factor 10
Progress:  0.41033067825247405
Inflation factor 10
Progress:  0.4344677769732078
Inflation factor 10
Progress:  0.4586048756939416
Inflation factor 10
Pro

In [46]:
np.array(raw_ds['image_stack']).shape, np.array(raw_ds['mask']).shape

((41430, 41, 2, 64), (41430, 64))

In [49]:
n_samples = int(inflation_factor * samples)
image_dims = (41, 2, target_size)
mask_dims = (target_size,)
chunk_size = 100

In [68]:
filepath = '/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/sentinel-1/extracted-640/train_data_inflated.h5' # note renamed files in directory!
if os.path.exists(filepath):
    os.remove(filepath)
    print('removed')
new_h5_file = h5py.File(filepath, 'w')

removed


In [69]:
new_h5_file.create_dataset("image_stack", shape=(n_samples,) + image_dims, chunks=(chunk_size,) + image_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['image_stack']))
new_h5_file.create_dataset("mask", shape=(n_samples,) + mask_dims, chunks=(chunk_size,) + mask_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['mask']))
new_h5_file.create_dataset("fid", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['fid']))
new_h5_file.create_dataset("label", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['label']))
new_h5_file.create_dataset("crop_name", shape=(n_samples,), chunks=(chunk_size,), dtype=h5py.string_dtype(), data=np.array(raw_ds['crop_name']))


<HDF5 dataset "crop_name": shape (41430,), type "|O">

In [70]:
new_h5_file.attrs['time_created'] = str(datetime.datetime.now())
new_h5_file.flush()
new_h5_file.close()

In [73]:
tmp = h5py.File(filepath, 'r') # check
print(tmp.keys())
print(tmp['image_stack'][0].shape, tmp['mask'][0].shape, tmp['label'][:10], tmp['fid'][:10], tmp['crop_name'][:10])

<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>
(41, 2, 64) (64,) [4 4 4 4 4 4 4 4 4 4] [80172 80172 80172 80172 80172 80172 80172 80172 80172 80172] [b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics']


## Sentinel-2, 640

In [3]:
assert False # stop automatic execution

In [25]:
train_S1_640 = h5py.File('/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/sentinel-2/extracted-640/train_data.h5', 'r')

In [26]:
samples = len(train_S1_640['label'])
print('Samples before inflation:', samples)

h5_keys = train_S1_640.keys()
print(h5_keys)

Samples before inflation: 4143
<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>


In [27]:
target_size = 64

In [28]:
raw_ds = {key:[] for key in h5_keys}
start_time = time.time()

for i in range(samples):
    this_crop_name = train_S1_640['crop_name'][i]
    this_fid       = train_S1_640['fid'][i]
    this_image_stack = train_S1_640['image_stack'][i]
    this_label       = train_S1_640['label'][i]
    this_mask        = train_S1_640['mask'][i]
    
    current_size = this_image_stack.shape[-1]
    
    inflation_factor = current_size // target_size
    
    #print('Inflation factor', inflation_factor)
    
    for j in range(inflation_factor):
        # always the same
        raw_ds['crop_name'].append(this_crop_name) 
        raw_ds['fid'].append(this_fid)
        raw_ds['label'].append(this_label)
        # divide
        raw_ds['image_stack'].extend(this_image_stack[np.newaxis, :, :, j*target_size:(j+1)*target_size])
        raw_ds['mask'].extend(this_mask[np.newaxis, j*target_size:(j+1)*target_size])
    
    if i%(samples//100)==0:
        print(f'Progress: { i/samples * 100:.1f}%, {(time.time() - start_time) / 60:.1f} min')
        start_time = time.time()

Progress: 0.0%, 0.0 min
Progress: 1.0%, 0.1 min
Progress: 2.0%, 0.1 min
Progress: 3.0%, 0.1 min
Progress: 4.0%, 0.1 min
Progress: 4.9%, 0.1 min
Progress: 5.9%, 0.1 min
Progress: 6.9%, 0.1 min
Progress: 7.9%, 0.1 min
Progress: 8.9%, 0.1 min
Progress: 9.9%, 0.1 min
Progress: 10.9%, 0.1 min
Progress: 11.9%, 0.1 min
Progress: 12.9%, 0.1 min
Progress: 13.9%, 0.1 min
Progress: 14.8%, 0.1 min
Progress: 15.8%, 0.1 min
Progress: 16.8%, 0.1 min
Progress: 17.8%, 0.1 min
Progress: 18.8%, 0.1 min
Progress: 19.8%, 0.1 min
Progress: 20.8%, 0.1 min
Progress: 21.8%, 0.1 min
Progress: 22.8%, 0.1 min
Progress: 23.8%, 0.1 min
Progress: 24.7%, 0.1 min
Progress: 25.7%, 0.1 min
Progress: 26.7%, 0.1 min
Progress: 27.7%, 0.1 min
Progress: 28.7%, 0.1 min
Progress: 29.7%, 0.1 min
Progress: 30.7%, 0.1 min
Progress: 31.7%, 0.1 min
Progress: 32.7%, 0.1 min
Progress: 33.6%, 0.1 min
Progress: 34.6%, 0.1 min
Progress: 35.6%, 0.1 min
Progress: 36.6%, 0.1 min
Progress: 37.6%, 0.1 min
Progress: 38.6%, 0.1 min
Progress: 3

In [46]:
np.array(raw_ds['image_stack']).shape, np.array(raw_ds['mask']).shape

((41430, 41, 2, 64), (41430, 64))

In [49]:
n_samples = int(inflation_factor * samples)
image_dims = (41, 2, target_size)
mask_dims = (target_size,)
chunk_size = 100

In [68]:
filepath = '/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/sentinel-1/extracted-640/train_data_inflated.h5' # note renamed files in directory!
if os.path.exists(filepath):
    os.remove(filepath)
    print('removed')
new_h5_file = h5py.File(filepath, 'w')

removed


In [69]:
new_h5_file.create_dataset("image_stack", shape=(n_samples,) + image_dims, chunks=(chunk_size,) + image_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['image_stack']))
new_h5_file.create_dataset("mask", shape=(n_samples,) + mask_dims, chunks=(chunk_size,) + mask_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['mask']))
new_h5_file.create_dataset("fid", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['fid']))
new_h5_file.create_dataset("label", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['label']))
new_h5_file.create_dataset("crop_name", shape=(n_samples,), chunks=(chunk_size,), dtype=h5py.string_dtype(), data=np.array(raw_ds['crop_name']))


<HDF5 dataset "crop_name": shape (41430,), type "|O">

In [70]:
new_h5_file.attrs['time_created'] = str(datetime.datetime.now())
new_h5_file.flush()
new_h5_file.close()

In [73]:
tmp = h5py.File(filepath, 'r') # check
print(tmp.keys())
print(tmp['image_stack'][0].shape, tmp['mask'][0].shape, tmp['label'][:10], tmp['fid'][:10], tmp['crop_name'][:10])

<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>
(41, 2, 64) (64,) [4 4 4 4 4 4 4 4 4 4] [80172 80172 80172 80172 80172 80172 80172 80172 80172 80172] [b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics']


## Planet, 640

In [3]:
assert False # stop automatic execution

In [21]:
train_P_640 = h5py.File('/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/planet/extracted-640/before_inflation_train_data.h5', 'r')

In [22]:
samples = len(train_P_640['label'])
print('Samples before inflation:', samples)

h5_keys = train_P_640.keys()
print(h5_keys)

Samples before inflation: 4143
<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>


In [23]:
target_size = 64

In [24]:
raw_ds = {key:[] for key in h5_keys}

for i in range(samples):
    this_crop_name = train_P_640['crop_name'][i]
    this_fid       = train_P_640['fid'][i]
    this_image_stack = train_P_640['image_stack'][i]
    this_label       = train_P_640['label'][i]
    this_mask        = train_P_640['mask'][i]
    
    current_size = this_image_stack.shape[-1]
    
    inflation_factor = current_size // target_size
    
    print('Inflation factor', inflation_factor)
    
    for j in range(inflation_factor):
        # always the same
        raw_ds['crop_name'].append(this_crop_name) 
        raw_ds['fid'].append(this_fid)
        raw_ds['label'].append(this_label)
        # divide
        raw_ds['image_stack'].extend(this_image_stack[np.newaxis, :, :, j*target_size:(j+1)*target_size])
        raw_ds['mask'].extend(this_mask[np.newaxis, j*target_size:(j+1)*target_size])
    
    if i%100:
        print('Progress: ', i/samples * 100)

Inflation factor 10
Inflation factor 10
Progress:  0.02413709872073377
Inflation factor 10
Progress:  0.04827419744146754
Inflation factor 10
Progress:  0.0724112961622013
Inflation factor 10
Progress:  0.09654839488293508
Inflation factor 10
Progress:  0.12068549360366883
Inflation factor 10
Progress:  0.1448225923244026
Inflation factor 10
Progress:  0.1689596910451364
Inflation factor 10
Progress:  0.19309678976587016
Inflation factor 10
Progress:  0.2172338884866039
Inflation factor 10
Progress:  0.24137098720733766
Inflation factor 10
Progress:  0.26550808592807146
Inflation factor 10
Progress:  0.2896451846488052
Inflation factor 10
Progress:  0.313782283369539
Inflation factor 10
Progress:  0.3379193820902728
Inflation factor 10
Progress:  0.3620564808110065
Inflation factor 10
Progress:  0.3861935795317403
Inflation factor 10
Progress:  0.41033067825247405
Inflation factor 10
Progress:  0.4344677769732078
Inflation factor 10
Progress:  0.4586048756939416
Inflation factor 10
Pro

KeyboardInterrupt: 

In [7]:
np.array(raw_ds['image_stack']).shape, np.array(raw_ds['mask']).shape

((41430, 244, 4, 64), (41430, 64))

In [11]:
n_samples = int(inflation_factor * samples)
image_dims = (244, 4, target_size)
mask_dims = (target_size,)
chunk_size = 100

In [12]:
filepath = '/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/planet/extracted-640/train_data.h5'
if os.path.exists(filepath):
    os.remove(filepath)
    print('removed')
new_h5_file = h5py.File(filepath, 'w')

removed


In [13]:
new_h5_file.create_dataset("image_stack", shape=(n_samples,) + image_dims, chunks=(chunk_size,) + image_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['image_stack']))
new_h5_file.create_dataset("mask", shape=(n_samples,) + mask_dims, chunks=(chunk_size,) + mask_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['mask']))
new_h5_file.create_dataset("fid", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['fid']))
new_h5_file.create_dataset("label", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['label']))
new_h5_file.create_dataset("crop_name", shape=(n_samples,), chunks=(chunk_size,), dtype=h5py.string_dtype(), data=np.array(raw_ds['crop_name']))


<HDF5 dataset "crop_name": shape (41430,), type "|O">

In [14]:
new_h5_file.attrs['time_created'] = str(datetime.datetime.now())
new_h5_file.flush()
new_h5_file.close()

In [15]:
tmp = h5py.File(filepath, 'r') # check
print(tmp.keys())
print(tmp['image_stack'][0].shape, tmp['mask'][0].shape, tmp['label'][:10], tmp['fid'][:10], tmp['crop_name'][:10])

<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>
(244, 4, 64) (64,) [4 4 4 4 4 4 4 4 4 4] [80172 80172 80172 80172 80172 80172 80172 80172 80172 80172] [b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics']


## Planet-5, 640

In [3]:
assert False # stop automatic execution

In [2]:
train_P_640 = h5py.File('/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/planet-5/extracted-640/train_data.h5', 'r')

In [3]:
samples = len(train_P_640['label'])
print('Samples before inflation:', samples)

h5_keys = train_P_640.keys()
print(h5_keys)

Samples before inflation: 4143
<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>


In [4]:
target_size = 64

In [6]:
raw_ds = {key:[] for key in h5_keys}

for i in range(samples):
    this_crop_name = train_P_640['crop_name'][i]
    this_fid       = train_P_640['fid'][i]
    this_image_stack = train_P_640['image_stack'][i]
    this_label       = train_P_640['label'][i]
    this_mask        = train_P_640['mask'][i]
    
    current_size = this_image_stack.shape[-1]
    
    inflation_factor = current_size // target_size
        
    for j in range(inflation_factor):
        # always the same
        raw_ds['crop_name'].append(this_crop_name) 
        raw_ds['fid'].append(this_fid)
        raw_ds['label'].append(this_label)
        # divide
        raw_ds['image_stack'].extend(this_image_stack[np.newaxis, :, :, j*target_size:(j+1)*target_size])
        raw_ds['mask'].extend(this_mask[np.newaxis, j*target_size:(j+1)*target_size])
    
    if i%100 == 0:
        print('Progress: ', i/samples * 100)

Progress:  0.0
Progress:  2.4137098720733765
Progress:  36.20564808110065
Progress:  38.619357953174024
Progress:  41.03306782524741
Progress:  43.44677769732078
Progress:  45.86048756939416
Progress:  48.274197441467535
Progress:  50.68790731354091
Progress:  53.101617185614295
Progress:  55.51532705768767
Progress:  57.92903692976105
Progress:  60.34274680183442
Progress:  62.7564566739078
Progress:  65.17016654598117
Progress:  67.58387641805454
Progress:  69.99758629012793
Progress:  72.4112961622013
Progress:  74.82500603427468
Progress:  77.23871590634805
Progress:  79.65242577842143
Progress:  82.06613565049481
Progress:  84.47984552256818
Progress:  86.89355539464157
Progress:  89.30726526671494
Progress:  91.72097513878832
Progress:  94.1346850108617
Progress:  96.54839488293507
Progress:  98.96210475500845


In [7]:
np.array(raw_ds['image_stack']).shape, np.array(raw_ds['mask']).shape

((41430, 48, 4, 64), (41430, 64))

In [12]:
n_samples = int(inflation_factor * samples)
image_dims = (48, 4, target_size)
mask_dims = (target_size,)
chunk_size = 100

In [13]:
filepath = '/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/planet-5/extracted-640/train_data_inflated.h5'
if os.path.exists(filepath):
    os.remove(filepath)
    print('removed')
new_h5_file = h5py.File(filepath, 'w')

removed


In [14]:
new_h5_file.create_dataset("image_stack", shape=(n_samples,) + image_dims, chunks=(chunk_size,) + image_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['image_stack']))
new_h5_file.create_dataset("mask", shape=(n_samples,) + mask_dims, chunks=(chunk_size,) + mask_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['mask']))
new_h5_file.create_dataset("fid", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['fid']))
new_h5_file.create_dataset("label", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['label']))
new_h5_file.create_dataset("crop_name", shape=(n_samples,), chunks=(chunk_size,), dtype=h5py.string_dtype(), data=np.array(raw_ds['crop_name']))


<HDF5 dataset "crop_name": shape (41430,), type "|O">

In [15]:
new_h5_file.attrs['time_created'] = str(datetime.datetime.now())
new_h5_file.flush()
new_h5_file.close()

In [16]:
tmp = h5py.File(filepath, 'r') # check
print(tmp.keys())
print(tmp['image_stack'][0].shape, tmp['mask'][0].shape, tmp['label'][:10], tmp['fid'][:10], tmp['crop_name'][:10])

<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>
(48, 4, 64) (64,) [4 4 4 4 4 4 4 4 4 4] [80172 80172 80172 80172 80172 80172 80172 80172 80172 80172] [b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics']


## Sentinel-1, 640, Germany

In [3]:
assert False # stop automatic execution

In [2]:
train_S1_640 = h5py.File('/work/ka1176/shared_data/2021-ai4food/dev_data/germany/sentinel-1/extracted-640/train_data.h5', 'r')

In [3]:
samples = len(train_S1_640['label'])
print('Samples before inflation:', samples)

h5_keys = train_S1_640.keys()
print(h5_keys)

Samples before inflation: 2504
<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>


In [4]:
target_size = 64

In [5]:
raw_ds = {key:[] for key in h5_keys}

for i in range(samples):
    this_crop_name = train_S1_640['crop_name'][i]
    this_fid       = train_S1_640['fid'][i]
    this_image_stack = train_S1_640['image_stack'][i]
    this_label       = train_S1_640['label'][i]
    this_mask        = train_S1_640['mask'][i]
    
    current_size = this_image_stack.shape[-1]
    
    inflation_factor = current_size // target_size
    
    print('Inflation factor', inflation_factor)
    
    for j in range(inflation_factor):
        # always the same
        raw_ds['crop_name'].append(this_crop_name) 
        raw_ds['fid'].append(this_fid)
        raw_ds['label'].append(this_label)
        # divide
        raw_ds['image_stack'].extend(this_image_stack[np.newaxis, :, :, j*target_size:(j+1)*target_size])
        raw_ds['mask'].extend(this_mask[np.newaxis, j*target_size:(j+1)*target_size])
    
    if i%samples//10 == 0:
        print('Progress: ', i/samples * 100)

Inflation factor 10
Progress:  0.0
Inflation factor 10
Progress:  0.039936102236421724
Inflation factor 10
Progress:  0.07987220447284345
Inflation factor 10
Progress:  0.11980830670926518
Inflation factor 10
Progress:  0.1597444089456869
Inflation factor 10
Progress:  0.1996805111821086
Inflation factor 10
Progress:  0.23961661341853036
Inflation factor 10
Progress:  0.27955271565495204
Inflation factor 10
Progress:  0.3194888178913738
Inflation factor 10
Progress:  0.35942492012779553
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflation factor 10
Inflatio

In [29]:
np.array(raw_ds['image_stack']).shape, np.array(raw_ds['mask']).shape

((41430, 76, 13, 64), (41430, 64))

In [36]:
n_samples = int(inflation_factor * samples)
image_dims = (76, 13, target_size)
mask_dims = (target_size,)
chunk_size = 100

In [37]:
filepath = '/work/ka1176/shared_data/2021-ai4food/dev_data/germany/sentinel-2/extracted-640/train_data_inflated.h5' # note renamed files in directory!
if os.path.exists(filepath):
    os.remove(filepath)
    print('removed')
new_h5_file = h5py.File(filepath, 'w')

removed


In [38]:
new_h5_file.create_dataset("image_stack", shape=(n_samples,) + image_dims, chunks=(chunk_size,) + image_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['image_stack']))
new_h5_file.create_dataset("mask", shape=(n_samples,) + mask_dims, chunks=(chunk_size,) + mask_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['mask']))
new_h5_file.create_dataset("fid", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['fid']))
new_h5_file.create_dataset("label", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['label']))
new_h5_file.create_dataset("crop_name", shape=(n_samples,), chunks=(chunk_size,), dtype=h5py.string_dtype(), data=np.array(raw_ds['crop_name']))


<HDF5 dataset "crop_name": shape (41430,), type "|O">

In [39]:
new_h5_file.attrs['time_created'] = str(datetime.datetime.now())
new_h5_file.flush()
new_h5_file.close()

In [40]:
tmp = h5py.File(filepath, 'r') # check
print(tmp.keys())
print(tmp['image_stack'][0].shape, tmp['mask'][0].shape, tmp['label'][:10], tmp['fid'][:10], tmp['crop_name'][:10])
print(len(tmp['image_stack']))

<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>
(76, 13, 64) (64,) [4 4 4 4 4 4 4 4 4 4] [80172 80172 80172 80172 80172 80172 80172 80172 80172 80172] [b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics']
41430


## Planet, 640

In [12]:
assert False # stop automatic execution

AssertionError: 

In [13]:
train_P_640 = h5py.File('/work/ka1176/shared_data/2021-ai4food/dev_data/germany/planet/extracted-640/train_data.h5', 'r')

In [14]:
samples = len(train_P_640['label'])
print('Samples before inflation:', samples)

h5_keys = train_P_640.keys()
print(h5_keys)

Samples before inflation: 2504
<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>


In [15]:
target_size = 64

In [16]:
raw_ds = {key:[] for key in h5_keys}

for i in range(samples):
    this_crop_name = train_P_640['crop_name'][i]
    this_fid       = train_P_640['fid'][i]
    this_image_stack = train_P_640['image_stack'][i]
    this_label       = train_P_640['label'][i]
    this_mask        = train_P_640['mask'][i]
    
    current_size = this_image_stack.shape[-1]
    
    inflation_factor = current_size // target_size
    
    print('Inflation factor', inflation_factor)
    
    for j in range(inflation_factor):
        # always the same
        raw_ds['crop_name'].append(this_crop_name) 
        raw_ds['fid'].append(this_fid)
        raw_ds['label'].append(this_label)
        # divide
        raw_ds['image_stack'].extend(this_image_stack[np.newaxis, :, :, j*target_size:(j+1)*target_size])
        raw_ds['mask'].extend(this_mask[np.newaxis, j*target_size:(j+1)*target_size])
    
    if i%100:
        print('Progress: ', i/samples * 100)

Inflation factor 10
Inflation factor 10
Progress:  0.039936102236421724
Inflation factor 10
Progress:  0.07987220447284345
Inflation factor 10
Progress:  0.11980830670926518
Inflation factor 10
Progress:  0.1597444089456869
Inflation factor 10
Progress:  0.1996805111821086
Inflation factor 10
Progress:  0.23961661341853036
Inflation factor 10
Progress:  0.27955271565495204
Inflation factor 10
Progress:  0.3194888178913738
Inflation factor 10
Progress:  0.35942492012779553
Inflation factor 10
Progress:  0.3993610223642172
Inflation factor 10
Progress:  0.43929712460063897
Inflation factor 10
Progress:  0.4792332268370607
Inflation factor 10
Progress:  0.5191693290734825
Inflation factor 10
Progress:  0.5591054313099041
Inflation factor 10
Progress:  0.5990415335463258
Inflation factor 10
Progress:  0.6389776357827476
Inflation factor 10
Progress:  0.6789137380191693
Inflation factor 10
Progress:  0.7188498402555911
Inflation factor 10
Progress:  0.7587859424920127
Inflation factor 10
Pr

In [17]:
np.array(raw_ds['image_stack']).shape, np.array(raw_ds['mask']).shape

((25040, 365, 4, 64), (25040, 64))

In [18]:
n_samples = int(inflation_factor * samples)
image_dims = (365, 4, target_size)
mask_dims = (target_size,)
chunk_size = 100

In [19]:
filepath = '/work/ka1176/shared_data/2021-ai4food/dev_data/germany/planet/extracted-640/train_data_inflated.h5'
if os.path.exists(filepath):
    os.remove(filepath)
    print('removed')
new_h5_file = h5py.File(filepath, 'w')

In [20]:
new_h5_file.create_dataset("image_stack", shape=(n_samples,) + image_dims, chunks=(chunk_size,) + image_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['image_stack']))
new_h5_file.create_dataset("mask", shape=(n_samples,) + mask_dims, chunks=(chunk_size,) + mask_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['mask']))
new_h5_file.create_dataset("fid", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['fid']))
new_h5_file.create_dataset("label", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['label']))
new_h5_file.create_dataset("crop_name", shape=(n_samples,), chunks=(chunk_size,), dtype=h5py.string_dtype(), data=np.array(raw_ds['crop_name']))


<HDF5 dataset "crop_name": shape (25040,), type "|O">

In [21]:
new_h5_file.attrs['time_created'] = str(datetime.datetime.now())
new_h5_file.flush()
new_h5_file.close()

In [22]:
tmp = h5py.File(filepath, 'r') # check
print(tmp.keys())
print(tmp['image_stack'][0].shape, tmp['mask'][0].shape, tmp['label'][:10], tmp['fid'][:10], tmp['crop_name'][:10])

<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>
(365, 4, 64) (64,) [9 9 9 9 9 9 9 9 9 9] [54 54 54 54 54 54 54 54 54 54] [b'Forage Crops' b'Forage Crops' b'Forage Crops' b'Forage Crops'
 b'Forage Crops' b'Forage Crops' b'Forage Crops' b'Forage Crops'
 b'Forage Crops' b'Forage Crops']


## Sentinel-1, 6400

In [3]:
assert False # stop automatic execution

In [41]:
train_S1_640 = h5py.File('/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/sentinel-1/extracted-6400/train_data.h5', 'r')

In [42]:
samples = len(train_S1_640['label'])
print('Samples before inflation:', samples)

h5_keys = train_S1_640.keys()
print(h5_keys)

Samples before inflation: 4143
<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>


In [43]:
target_size = 64

In [44]:
raw_ds = {key:[] for key in h5_keys}

for i in range(samples):
    this_crop_name = train_S1_640['crop_name'][i]
    this_fid       = train_S1_640['fid'][i]
    this_image_stack = train_S1_640['image_stack'][i]
    this_label       = train_S1_640['label'][i]
    this_mask        = train_S1_640['mask'][i]
    
    current_size = this_image_stack.shape[-1]
    
    inflation_factor = min(max_inflation_factor, current_size // target_size)
    
    #print('Inflation factor', inflation_factor)
    
    for j in range(inflation_factor):
        # always the same
        raw_ds['crop_name'].append(this_crop_name) 
        raw_ds['fid'].append(this_fid)
        raw_ds['label'].append(this_label)
        # divide
        raw_ds['image_stack'].extend(this_image_stack[np.newaxis, :, :, j*target_size:(j+1)*target_size])
        raw_ds['mask'].extend(this_mask[np.newaxis, j*target_size:(j+1)*target_size])
    
    if i%(samples//100)==0:
        print(f'Progress: { i/samples * 100:.1f}%, {(time.time() - start_time) / 60:.1f} min')
        start_time = time.time()

Progress: 0.0%, 6.0 min
Progress: 1.0%, 0.1 min
Progress: 2.0%, 0.1 min
Progress: 3.0%, 0.1 min
Progress: 4.0%, 0.1 min
Progress: 4.9%, 0.1 min
Progress: 5.9%, 0.1 min
Progress: 6.9%, 0.1 min
Progress: 7.9%, 0.1 min
Progress: 8.9%, 0.1 min
Progress: 9.9%, 0.1 min
Progress: 10.9%, 0.1 min
Progress: 11.9%, 0.1 min
Progress: 12.9%, 0.1 min
Progress: 13.9%, 0.1 min
Progress: 14.8%, 0.1 min
Progress: 15.8%, 0.1 min
Progress: 16.8%, 0.1 min
Progress: 17.8%, 0.1 min
Progress: 18.8%, 0.1 min
Progress: 19.8%, 0.1 min
Progress: 20.8%, 0.1 min
Progress: 21.8%, 0.1 min
Progress: 22.8%, 0.1 min
Progress: 23.8%, 0.1 min
Progress: 24.7%, 0.1 min
Progress: 25.7%, 0.1 min
Progress: 26.7%, 0.1 min
Progress: 27.7%, 0.1 min
Progress: 28.7%, 0.1 min
Progress: 29.7%, 0.1 min
Progress: 30.7%, 0.1 min
Progress: 31.7%, 0.1 min
Progress: 32.7%, 0.1 min
Progress: 33.6%, 0.1 min
Progress: 34.6%, 0.1 min
Progress: 35.6%, 0.1 min
Progress: 36.6%, 0.1 min
Progress: 37.6%, 0.1 min
Progress: 38.6%, 0.1 min
Progress: 3

In [45]:
np.array(raw_ds['image_stack']).shape, np.array(raw_ds['mask']).shape

((103575, 41, 2, 64), (103575, 64))

In [46]:
n_samples = int(inflation_factor * samples)
image_dims = (41, 2, target_size)
mask_dims = (target_size,)
chunk_size = 100

In [47]:
filepath = '/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/sentinel-1/extracted-6400/train_data_inflated.h5' # note renamed files in directory!
if os.path.exists(filepath):
    os.remove(filepath)
    print('removed')
new_h5_file = h5py.File(filepath, 'w')

removed


In [48]:
new_h5_file.create_dataset("image_stack", shape=(n_samples,) + image_dims, chunks=(chunk_size,) + image_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['image_stack']))
new_h5_file.create_dataset("mask", shape=(n_samples,) + mask_dims, chunks=(chunk_size,) + mask_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['mask']))
new_h5_file.create_dataset("fid", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['fid']))
new_h5_file.create_dataset("label", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['label']))
new_h5_file.create_dataset("crop_name", shape=(n_samples,), chunks=(chunk_size,), dtype=h5py.string_dtype(), data=np.array(raw_ds['crop_name']))


<HDF5 dataset "crop_name": shape (103575,), type "|O">

In [49]:
new_h5_file.attrs['time_created'] = str(datetime.datetime.now())
new_h5_file.flush()
new_h5_file.close()

In [50]:
tmp = h5py.File(filepath, 'r') # check
print(tmp.keys())
print(tmp['image_stack'][0].shape, tmp['mask'][0].shape, tmp['label'][:10], tmp['fid'][:10], tmp['crop_name'][:10])

<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>
(41, 2, 64) (64,) [4 4 4 4 4 4 4 4 4 4] [80172 80172 80172 80172 80172 80172 80172 80172 80172 80172] [b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics']


## Planet, 6400

In [2]:
assert False # stop automatic execution

AssertionError: 

In [3]:
train_P_640 = h5py.File('/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/planet/extracted-6400/train_data.h5', 'r')

In [4]:
samples = len(train_P_640['label'])
print('Samples before inflation:', samples)

h5_keys = train_P_640.keys()
print(h5_keys)

Samples before inflation: 4143
<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>


In [5]:
target_size = 64

In [6]:
import time

In [7]:
max_inflation_factor = 25 # set to large value to keep all

In [9]:
raw_ds = {key:[] for key in h5_keys}

start_time = time.time()

for i in range(samples):
    
    this_crop_name = train_P_640['crop_name'][i]
    this_fid       = train_P_640['fid'][i]
    this_image_stack = train_P_640['image_stack'][i]
    this_label       = train_P_640['label'][i]
    this_mask        = train_P_640['mask'][i]
    
    current_size = this_image_stack.shape[-1]
    
    inflation_factor = min(max_inflation_factor, current_size // target_size)
    
    #print('Inflation factor', inflation_factor)
    
    for j in range(inflation_factor):
        # always the same
        raw_ds['crop_name'].append(this_crop_name) 
        raw_ds['fid'].append(this_fid)
        raw_ds['label'].append(this_label)
        # divide
        raw_ds['image_stack'].extend(this_image_stack[np.newaxis, :, :, j*target_size:(j+1)*target_size])
        raw_ds['mask'].extend(this_mask[np.newaxis, j*target_size:(j+1)*target_size])
    
    if i%(samples//100)==0:
        print(f'Progress: { i/samples * 100:.1f}%, {(time.time() - start_time) / 60:.1f} min')
        start_time = time.time()

Progress: 0.0%, 0.0 min
Progress: 1.0%, 1.5 min
Progress: 2.0%, 1.5 min
Progress: 3.0%, 1.5 min
Progress: 4.0%, 1.4 min
Progress: 4.9%, 1.5 min
Progress: 5.9%, 1.5 min
Progress: 6.9%, 1.5 min
Progress: 7.9%, 1.5 min
Progress: 8.9%, 1.4 min
Progress: 9.9%, 1.5 min
Progress: 10.9%, 1.5 min
Progress: 11.9%, 1.5 min
Progress: 12.9%, 1.5 min
Progress: 13.9%, 1.4 min
Progress: 14.8%, 1.5 min
Progress: 15.8%, 1.5 min
Progress: 16.8%, 1.5 min
Progress: 17.8%, 1.5 min
Progress: 18.8%, 1.4 min
Progress: 19.8%, 1.5 min
Progress: 20.8%, 1.4 min
Progress: 21.8%, 1.5 min
Progress: 22.8%, 1.4 min
Progress: 23.8%, 1.4 min
Progress: 24.7%, 1.5 min
Progress: 25.7%, 1.5 min
Progress: 26.7%, 1.5 min
Progress: 27.7%, 1.5 min
Progress: 28.7%, 1.5 min
Progress: 29.7%, 1.5 min
Progress: 30.7%, 1.5 min
Progress: 31.7%, 1.5 min
Progress: 32.7%, 1.4 min
Progress: 33.6%, 1.4 min
Progress: 34.6%, 1.5 min
Progress: 35.6%, 1.5 min
Progress: 36.6%, 1.5 min
Progress: 37.6%, 1.5 min
Progress: 38.6%, 1.5 min
Progress: 3

In [10]:
import pickle
with open('raw_ds_planet_south-africa_6400.pkl', 'wb') as pf:
    pickle.dump(raw_ds, pf)

In [8]:
np.array(raw_ds['image_stack']).shape, np.array(raw_ds['mask']).shape

MemoryError: Unable to allocate 96.4 GiB for an array with shape (414300, 244, 4, 64) and data type float32

In [11]:
n_samples = int(inflation_factor * samples)
image_dims = (244, 4, target_size)
mask_dims = (target_size,)
chunk_size = 100

In [16]:
filepath = '/work/ka1176/shared_data/2021-ai4food/dev_data/south-africa/planet/extracted-6400/train_data_inflated.h5'
if os.path.exists(filepath):
    os.remove(filepath)
    print('removed')
new_h5_file = h5py.File(filepath, 'w')

In [17]:
new_h5_file.create_dataset("image_stack", shape=(n_samples,) + image_dims, chunks=(chunk_size,) + image_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['image_stack']))
new_h5_file.create_dataset("mask", shape=(n_samples,) + mask_dims, chunks=(chunk_size,) + mask_dims, fletcher32=True, dtype='float32', data=np.array(raw_ds['mask']))
new_h5_file.create_dataset("fid", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['fid']))
new_h5_file.create_dataset("label", shape=(n_samples,), chunks=(chunk_size,), fletcher32=True, dtype='int', data=np.array(raw_ds['label']))
new_h5_file.create_dataset("crop_name", shape=(n_samples,), chunks=(chunk_size,), dtype=h5py.string_dtype(), data=np.array(raw_ds['crop_name']))


<HDF5 dataset "crop_name": shape (103575,), type "|O">

In [18]:
new_h5_file.attrs['time_created'] = str(datetime.datetime.now())
new_h5_file.flush()
new_h5_file.close()

In [19]:
tmp = h5py.File(filepath, 'r') # check
print(tmp.keys())
print(tmp['image_stack'][0].shape, tmp['mask'][0].shape, tmp['label'][:10], tmp['fid'][:10], tmp['crop_name'][:10])

<KeysViewHDF5 ['crop_name', 'fid', 'image_stack', 'label', 'mask']>
(244, 4, 64) (64,) [4 4 4 4 4 4 4 4 4 4] [80172 80172 80172 80172 80172 80172 80172 80172 80172 80172] [b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics' b'Lucerne/Medics'
 b'Lucerne/Medics' b'Lucerne/Medics']


In [20]:
len(tmp['image_stack'])

103575